In [5]:
 # This is Notebook for File Type detection using convolutional neural network on TREC-DD dataset
 # The data used is in 256 byte frequencies format
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import pickle
from utility import *
from sklearn.metrics import classification_report
tf.logging.set_verbosity(tf.logging.INFO)

In [6]:
def cnn_model_fn(features, labels, mode, params):
  """Model function for CNN."""
  
  config = params

  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  
  input_layer = tf.reshape(features["x"], [-1, 256, 1, 1])
  print(input_layer.shape)
  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 256, 1, 1]
  # Output Tensor Shape: [batch_size, 256, 1, 32]
  # kernel_size specifies [width, height]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[config['kernel1_width'], 1],
      padding="same",
      activation=tf.nn.relu)
  print(conv1.shape)
  normal_1 = tf.layers.batch_normalization(conv1, training=mode == tf.estimator.ModeKeys.TRAIN)
    
  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 256, 1, 32]
  # Output Tensor Shape: [batch_size, 128, 1, 32]
  # pool_size, strides [width, height]
  pool1 = tf.layers.max_pooling2d(inputs=normal_1, pool_size=[config['pool1_width'], 1], strides=[config['pool1_stride'],1])
  print(pool1.shape)
  # Convolutional Layer #2
  # Computes 64 features using a 5x1 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 128, 1, 32]
  # Output Tensor Shape: [batch_size, 128, 1, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[config['kernel2_width'], 1],
      padding="same",
      activation=tf.nn.relu)
  print(conv2.shape)
  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 128, 1, 64]
  # Output Tensor Shape: [batch_size, 64, 1, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[config['pool2_width'], 1], strides=[config['pool2_stride'],1])
  print(pool2.shape)
  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 64, 1, 64]
  # Output Tensor Shape: [batch_size, 64 * 1* 64]]
  pool2_shape = pool2.shape
  pool2_flat = tf.reshape(pool2, [-1, pool2_shape[1] * pool2_shape[2] * pool2_shape[3]])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 64 * 1* 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=config['dense_units'], activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 93]
  logits = tf.layers.dense(inputs=dropout, units=config['nclasses'])

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=config['nclasses'])
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [7]:
unused_argv = ['pilot_1d_cnn_newAPI.py', 'data_1p_companded.csv', 'model/', 1024, 512, 256]

filename = unused_argv[1]

if os.path.isfile('ft_to_idx.npy') and os.path.isfile('idx_to_ft.npy') and os.path.isfile('nclasses.npy') and os.path.isfile('group_data.npy'):
    ft_to_idx = np.load('ft_to_idx.npy')
    ft_to_idx = ft_to_idx.item()
    idx_to_ft = np.load('idx_to_ft.npy')
    idx_to_ft = idx_to_ft.item()
    nclasses = np.load('nclasses.npy')
    #f = open('group_data.pkl','r')
    #group_data = pickle.load('group_data.pkl')
    group_data = np.load('group_data.npy') 
    #group_data = np.load('toy_data.npy') # for proof of algo purpose, real use case should use the above line
    group_data = group_data.item()

else:
    ft_to_idx, idx_to_ft, nclasses, group_data = prepare_file(filename)
    np.save("ft_to_idx", ft_to_idx)
    np.save("idx_to_ft", idx_to_ft)
    np.save("nclasses", nclasses)
    #f = open('group_data.pkl','w')
    #pickle.dump(group_data, f)
    np.save("group_data", group_data)

#gen train set
cnt = map(lambda x: len(x),group_data.values())
percentile_40 = np.percentile(cnt, 40)
train, dev ,test = train_dev_test_split(group_data, proportion = [0.6,0.2], thre = percentile_40) # should be 1000 or so
train_data, train_labels = gen_feed(train, ft_to_idx, upper_limit=10000)

dev_data, dev_labels = gen_feed(dev, ft_to_idx, upper_limit=3300)
test_data, test_labels = gen_feed(test, ft_to_idx, upper_limit=3300)
train_data = train_data.astype(np.float32) 
dev_data = dev_data.astype(np.float32)
test_data = test_data.astype(np.float32) 
del train,dev,test, group_data



In [11]:
from itertools import product
dense_units = [1024, 512, 256, 128]
batch_sizes = [64]#,128]#,256,512]
dropouts = [0.4]#,0.2]
kernel_width = [3]#,5,7]
pool_width = [2]#,4]
pool_stride = [2]#,4]


#reset config, defined in previous block
config = {}
config['nclasses'] =  len(np.unique(train_labels))
results = []
for dense_unit,batch_size,dropout,kernel_width,pool_width,pool_stride in product(dense_units,batch_sizes,dropouts,kernel_width,pool_width,pool_stride):
    config['dropout'] = dropout
    config['dense_units'] = dense_unit
    config['kernel1_width'] = kernel_width
    config['kernel2_width'] = kernel_width
    config['pool1_width'] = pool_width
    config['pool2_width'] = pool_width
    config['pool1_stride'] = pool_stride
    config['pool2_stride'] = pool_stride
    
    # Create the Estimator
    cnn_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir=None, params=config)

    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=5000) 

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=batch_size,
      num_epochs=3,
      shuffle=True)

    steps = train_data.shape[0]/batch_size
    cnn_classifier.train(
      input_fn=train_input_fn,
      steps=None, # 60000
      hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": dev_data},
        y=dev_labels,
        num_epochs=1,
        shuffle=False)

    eval_results = cnn_classifier.evaluate(input_fn=eval_input_fn)
    print ('\n----setting----')
    print('{0:12} ==> {1:5d}'.format('batch_size', batch_size))
    for k,v in config.iteritems():
        print('{0:12} ==> {1:5f}'.format(k, v))  
    print ('----performance----')
    print(eval_results)
    print ('\n')
    results.append((eval_results['accuracy'], batch_size, dropout, dense_unit, kernel_width, pool_width, pool_stride))
    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1195a4ad0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpKxSSKb', '_save_summary_steps': 100}
(?, 256, 1, 1)
(?, 256, 1, 32)
(?, 128, 1, 32)
(?, 128, 1, 64)
(?, 64, 1, 64)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpKxSSKb/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.02154318  0.01734272  0.00739854 ...,  0.0098566   0.01372289
   0.00961685]
 [ 0.02868428  0.01057385  0.

In [12]:
results = sorted(results, key=lambda x: x[0], reverse=True)

In [14]:
best_param = results[0]
batch_size = best_param[1]
config['dropout'] = best_param[2]
config['dense_units'] = best_param[3]
config['kernel1_width'] = best_param[4]
config['kernel2_width'] = best_param[4]
config['pool1_width'] = best_param[5]
config['pool2_width'] = best_param[5]
config['pool1_stride'] = best_param[6]
config['pool2_stride'] = best_param[6]

# Create the Estimator
cnn_classifier = tf.estimator.Estimator(
model_fn=cnn_model_fn, model_dir='model/cnn_1d_1part', params=config)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=5000) 

#make full training data
train_full_data = np.vstack((train_data,dev_data))
train_full_labels = np.hstack((train_labels,dev_labels))

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": train_full_data},
  y=train_full_labels,
  batch_size=batch_size,
  num_epochs=3,
  shuffle=True)

steps = train_full_data.shape[0]/batch_size
cnn_classifier.train(
  input_fn=train_input_fn,
  steps=None, # 60000
  hooks=[logging_hook])

pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    num_epochs=1,
    shuffle=False)

predictions = cnn_classifier.predict(input_fn=pred_input_fn)
predictions = list(p["classes"] for p in predictions)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11959f4d0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'model/cnn_1d_1part', '_save_summary_steps': 100}
(?, 256, 1, 1)
(?, 256, 1, 32)
(?, 128, 1, 32)
(?, 128, 1, 64)
(?, 64, 1, 64)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into model/cnn_1d_1part/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.01134625  0.01962971  0.01066241 ...,  0.01765301  0.00353571
   0.0418691 ]
 [ 0.00606116  0.02025443  0.00322493 ...,  0.02283608  0.01543568
   0.0468316 ]
 [ 0.00875253  0.01688779  

ValueError: Unknown format code 'd' for object of type 'float'

In [16]:
print ('----Best setting----')
print('{0:12} ==> {1:5d}'.format('batch_size', batch_size))
for k,v in config.iteritems():
    print('{0:12} ==> {1:5f}'.format(k, v))  
print ('\n')

target_names = np.unique(test_labels)
target_names = np.sort(target_names)
target_names = [idx_to_ft[i] for i in target_names]
print (classification_report(test_labels, predictions, target_names=target_names))

----Best setting----
batch_size   ==>    64
dense_units  ==> 512.000000
kernel2_width ==> 3.000000
dropout      ==> 0.400000
pool2_width  ==> 2.000000
pool1_stride ==> 2.000000
nclasses     ==> 73.000000
pool2_stride ==> 2.000000
kernel1_width ==> 3.000000
pool1_width  ==> 2.000000


                                                                           precision    recall  f1-score   support

                                                                text/html       0.00      0.00      0.00      3300
                                                                image/png       0.00      0.00      0.00      3300
                                                      application/rss+xml       0.00      0.00      0.00      1511
                                                    application/xhtml+xml       0.00      0.00      0.00      3300
                                                          application/pdf       0.00      0.00      0.00      3300
                        

//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
from sklearn.metrics import accuracy_score
print (accuracy_score(test_labels, predictions,normalize=True))

0.108182533438
